In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [22]:
# For reproducibility
torch.manual_seed(1)

In [23]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [24]:
x_train.shape, y_train.shape

(torch.Size([8, 3]), torch.Size([8]))

In [25]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

In [26]:
x_test.shape, y_test.shape

(torch.Size([3, 3]), torch.Size([3]))

In [27]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
    def forward(self, x):
        return self.linear(x)

In [28]:
model = SoftmaxClassifierModel()

In [29]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [30]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.cross_entropy(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [31]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1] # t.max(dim=0): 특정 차원단위 최대값(max)와 최대값을 갖는 데이터 인덱스 인출
    
    # 점수가 최대인 것이 predicted class 
    
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

In [32]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [33]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


### Learning Rate
learning rate이 너무 크면 diverge 하면서 cost 가 점점 늘어난다 (overshooting).

In [34]:
model = SoftmaxClassifierModel()

In [35]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [36]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.750000
Epoch    2/20 Cost: 1279135.125000
Epoch    3/20 Cost: 1198379.125000
Epoch    4/20 Cost: 1098825.625000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.156250
Epoch    7/20 Cost: 1532260.000000
Epoch    8/20 Cost: 1651504.000000
Epoch    9/20 Cost: 521878.468750
Epoch   10/20 Cost: 1397263.125000
Epoch   11/20 Cost: 750986.250000
Epoch   12/20 Cost: 918691.500000
Epoch   13/20 Cost: 1487888.125000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.062500
Epoch   16/20 Cost: 1140048.750000
Epoch   17/20 Cost: 940566.562500
Epoch   18/20 Cost: 931638.187500
Epoch   19/20 Cost: 1971322.625000


In [37]:
model = SoftmaxClassifierModel()

In [38]:
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [39]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


### Data Preprocessing 
데이터 전처리 zero-center, normalize

In [42]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [43]:
# 평균
mu = x_train.mean(dim=0)

In [44]:
# 표준편차 
sigma = x_train.std(dim=0)

In [45]:
# normalization
norm_x_train = (x_train - mu) / sigma

In [46]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


성능차이 비교

In [68]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [63]:
model = MultivariateLinearRegressionModel()

In [64]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [65]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [61]:
# normalization의 성능이 높음
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29632.109375
Epoch    1/20 Cost: 18827.687500
Epoch    2/20 Cost: 12010.023438
Epoch    3/20 Cost: 7674.996094
Epoch    4/20 Cost: 4908.805664
Epoch    5/20 Cost: 3140.833496
Epoch    6/20 Cost: 2010.023438
Epoch    7/20 Cost: 1286.502563
Epoch    8/20 Cost: 823.503906
Epoch    9/20 Cost: 527.196472
Epoch   10/20 Cost: 337.560760
Epoch   11/20 Cost: 216.190262
Epoch   12/20 Cost: 138.509491
Epoch   13/20 Cost: 88.789742
Epoch   14/20 Cost: 56.965099
Epoch   15/20 Cost: 36.593456
Epoch   16/20 Cost: 23.551987
Epoch   17/20 Cost: 15.201983
Epoch   18/20 Cost: 9.854509
Epoch   19/20 Cost: 6.428822


In [66]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 39676.117188
Epoch    1/20 Cost: 768096993280.000000
Epoch    2/20 Cost: 14872840901251563520.000000
Epoch    3/20 Cost: 287986233493896198084362240.000000
Epoch    4/20 Cost: 5576344234862370436592726536355840.000000
Epoch    5/20 Cost: inf
Epoch    6/20 Cost: inf
Epoch    7/20 Cost: inf
Epoch    8/20 Cost: inf
Epoch    9/20 Cost: inf
Epoch   10/20 Cost: inf
Epoch   11/20 Cost: inf
Epoch   12/20 Cost: nan
Epoch   13/20 Cost: nan
Epoch   14/20 Cost: nan
Epoch   15/20 Cost: nan
Epoch   16/20 Cost: nan
Epoch   17/20 Cost: nan
Epoch   18/20 Cost: nan
Epoch   19/20 Cost: nan


### Overfitting
오버피팅을 방지하는 방법 
1. 더 많은 학습 데이터
2. 더 적은 양의 feature
3. Regularization

In [41]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm 계산
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)      # parm을 regularization
            
        cost += l2_reg

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))

In [69]:
model = MultivariateLinearRegressionModel()

In [70]:
optimizer = optim.SGD(model.parameters(), lr = 1e-1)

In [71]:
# overfitting을 줄여줌 - regularization
train_with_regularization(model, optimizer, norm_x_train, y_train)

Epoch    1/20 Cost: 29785.947266
Epoch    2/20 Cost: 18916.179688
Epoch    3/20 Cost: 12130.879883
Epoch    4/20 Cost: 7818.112305
Epoch    5/20 Cost: 5066.614258
Epoch    6/20 Cost: 3308.173584
Epoch    7/20 Cost: 2183.499023
Epoch    8/20 Cost: 1463.913086
Epoch    9/20 Cost: 1003.432861
Epoch   10/20 Cost: 708.736084
Epoch   11/20 Cost: 520.128723
Epoch   12/20 Cost: 399.414795
Epoch   13/20 Cost: 322.152069
Epoch   14/20 Cost: 272.698120
Epoch   15/20 Cost: 241.042297
Epoch   16/20 Cost: 220.777222
Epoch   17/20 Cost: 207.802582
Epoch   18/20 Cost: 199.494064
Epoch   19/20 Cost: 194.172195
Epoch   20/20 Cost: 190.761963
